In [39]:
import json 

with open('레지던스_reviews.json','r',encoding='utf-8')as f:
    review_list = json.load(f)

review_list[:5]

[{'review': '30층 오션뷰였는데 앉아있기만 해도 힐링 그 자체였습니다.\r\n1층까지 환승해서 가야되는 불편함 말고는 대만족입니다!\r\n출장으로 내려왔지만 다시오고싶습니다.',
  'stars': '5점',
  'date': '2024.04.12'},
 {'review': '뷰가 정말 좋아요!\r\n바로앞에 해운대바다고 1분거리에 해운대야시장\r\n위치적으로도 뷰도 최곱니다ㅋㅋㅋ\r\n그리고 직원분들도 너무 친절하세요!!!\r\n아빠모시고왔는데 만족해하셔서 기분이 좋네요~후기가 많이 갈려서 걱정했는데 청결했어요!',
  'stars': '5점',
  'date': '2024.05.05'},
 {'review': '엘리베이터 환승이 불편했지만\n객실내 깔끔하게 정리가 잘 되어 있고\n전망이 나쁘지 않아 좋았어요',
  'stars': '4점',
  'date': '3일 전'},
 {'review': '엄청 깨끄ㅅ하고 정갈함 단지 넓은만큼 욕실 원풀 설치 욕조가 없어 아쉽고 샤워부스도 좁아 아쉬웠다 바다 뷰는 낯,밤 모두 아름다웠다',
  'stars': '5점',
  'date': '2024.10.23'},
 {'review': '직원들이 굉장히 친절해서 급한 일로 업무처리 가능하게 도와주셔서 감사했습니다 침구가 좋아서 무엇보다 가족들이 모두 잠을 잘 잤습니다 건물 내 호텔이 3개라 정확히 확인하고 이용할 필요는 있습니다(엘리베이터환승 등) 6층에 프런트 있고 그 중에도 왼쪽이었습니다 모르고 지난 번엔 건물 내 다른 호텔 이용했었는데 이번에 이용한 해운대 씨클라우드 호텔 앤 레지던스가 더 좋았습니다 같은 건물이라도 어디서 어떻게 관리하는지에 따라 다르긴 하네요 잘 쉬고 갑니다',
  'stars': '5점',
  'date': '2024.09.21'}]

In [18]:
good_cnt, bad_cnt = 0, 0

for review in review_list:
    if review['stars'] == '5점':
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(227, 193)

In [19]:
reviews_good, reviews_bad = [], []

for review in review_list:
    if review['stars'] == '5점':
        reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

reviews_good[:5], reviews_bad[:5]

(['[REVIEW_START]30층 오션뷰였는데 앉아있기만 해도 힐링 그 자체였습니다.\r\n1층까지 환승해서 가야되는 불편함 말고는 대만족입니다!\r\n출장으로 내려왔지만 다시오고싶습니다.[REVIEW_END]',
  '[REVIEW_START]뷰가 정말 좋아요!\r\n바로앞에 해운대바다고 1분거리에 해운대야시장\r\n위치적으로도 뷰도 최곱니다ㅋㅋㅋ\r\n그리고 직원분들도 너무 친절하세요!!!\r\n아빠모시고왔는데 만족해하셔서 기분이 좋네요~후기가 많이 갈려서 걱정했는데 청결했어요![REVIEW_END]',
  '[REVIEW_START]엄청 깨끄ㅅ하고 정갈함 단지 넓은만큼 욕실 원풀 설치 욕조가 없어 아쉽고 샤워부스도 좁아 아쉬웠다 바다 뷰는 낯,밤 모두 아름다웠다[REVIEW_END]',
  '[REVIEW_START]직원들이 굉장히 친절해서 급한 일로 업무처리 가능하게 도와주셔서 감사했습니다 침구가 좋아서 무엇보다 가족들이 모두 잠을 잘 잤습니다 건물 내 호텔이 3개라 정확히 확인하고 이용할 필요는 있습니다(엘리베이터환승 등) 6층에 프런트 있고 그 중에도 왼쪽이었습니다 모르고 지난 번엔 건물 내 다른 호텔 이용했었는데 이번에 이용한 해운대 씨클라우드 호텔 앤 레지던스가 더 좋았습니다 같은 건물이라도 어디서 어떻게 관리하는지에 따라 다르긴 하네요 잘 쉬고 갑니다[REVIEW_END]',
  '[REVIEW_START]아빠모시고가는 여행이라 오션뷰와 접근성보고 예약했어요! 총3박4일 숙박했고 2604호 였어요~\n안좋은후기도 많았지만 제가 배정받은호수는 사진에 보이는 뷰 그대로였어요ㅎㅎ\n뷰도 최고고 깨끗했어요^^\n화장실도 좀 좁지만 있을건 다 있고 욕실슬리퍼도있고 괜찮았어요\n드라이기는 좀 약해서 제가 가져간 드라이기 사용했어요!\n다음에 부산가면 또 숙박하려구요~\n직원분도 엄청 친절해요^^\n제가 짐캐리로 짐운송서비스 신청했었는데 숙소에 도착했다고 직접 전화도 주셨어요ㅎㅎ\n매일 청소도 해주셨구 여튼 아빠도 만족하셨어요![REVIEW_END]']

In [20]:
import datetime
from dateutil import parser

def preprocess_reviews(path='레지던스_reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue

        if review['stars'] == '5점':
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)

    return review_good_text, review_bad_text

good, bad = preprocess_reviews("레지던스_reviews.json")

In [21]:
PROMPT_BASELINE = f"""아래 숙소에 대해서 3문장 내로 요약해줘:"""

In [40]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

In [41]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [42]:
reviews, _ = preprocess_reviews(path = "./레지던스_reviews.json")
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

해운대에 위치한 숙소로 깨끗하고 정갈한 객실을 제공하며, 바다 뷰가 아름다운 곳이지만 욕조가 없고 샤워부스가 좁은 점이 아쉬웠다는 후기가 있습니다. 직원들이 친절하고 침구가 편안하여 가족 모두 잘 쉬었다고 하며, 건물 내 호텔이 3개이므로 정확한 확인이 필요하다는 후기도 있습니다. 위치가 좋고 가격대비 시설이 좋아 가성비가 뛰어나다는 후기도 있습니다.


In [43]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE,temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

summary_real_20241003 = '교통이 편리한 위치에 있으며, 청결한 상태를 유지하고 있어 많은 후기가 긍정적입니다. 객실은 넷플릭스 시청을 위한 기기 대여가 필요하지만 전반적으로 깔끔하고 편안한 분위기를 제공합니다. 직원들의 친절한 응대가 인상적이며, 조식과 디너 서비스 역시 우수하다는 평이 있습니다. 다만, 방음과 시설 노후화 관련 아쉬운 점이 언급되기도 했습니다. 종합적으로 접근성과 서비스가 우수한 조건을 갖춘 숙소입니다.'

In [44]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt,b_cnt,draw_cnt=0,0,0 
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i]) 
        verdict_text = completion.choices[0].message.content 

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else: 
            print("Evaluation Error")

    return a_cnt, b_cnt, draw_cnt

wins, loses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20241003]*eval_count) 
print(f"Wins: {wins}, Loses: {loses}, Ties: {ties}")

100%|██████████| 10/10 [00:41<00:00,  4.14s/it]

Wins: 9, Loses: 1, Ties: 0


## One-Shot

In [45]:
prompt = "당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다."

In [46]:
reviews_1shot, _ = preprocess_reviews(path = "./레지던스_reviews.json") 
summary_1shot = summarize(reviews_1shot, prompt, temperature=1.0, model = 'gpt-4o-2024-05-13').choices[0].message.content


In [47]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

다음은 요약 예시입니다.
예시 요약 결과:
{reviews_1shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt_1shot, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(eval_count)])
print(f"Wins: {wins}, Loses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:42<00:00,  4.22s/it]

Wins: 10, Loses: 0, Ties: 0


## TWO - SHOT

In [ ]:
prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

다음은 요약 예시입니다.
좋은 리뷰 요약 결과:
{reviews_1shot}
나쁜 리뷰 요약 결과:
{summary_1shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""
eval_count = 10
summaries = [summarize(reviews, prompt_2shot, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(eval_count))])
print(f"Wins: {wins}, Loses: {losses}, Ties: {ties}")

100%|██████████| 10/10 [00:43<00:00,  4.30s/it]

Wins: 10, Loses: 0, Ties: 0
